In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt   ##绘图库
import seaborn as sns ##图片风格扩展
from datetime import timedelta  #时间
from sklearn.preprocessing import LabelEncoder  ##label编码
from matplotlib import font_manager  ##解决plot中文字符显示问题
plt.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体为新宋体。
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像时'-'显示为方块的问题。
from matplotlib import colors
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split   ## 划分
from sklearn.metrics import r2_score
import torch
## 代替keras库,LSTM模型
from torch import nn
from sklearn.preprocessing import MinMaxScaler

# 检查是否有可用的 GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # 使用GPU
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")  # 使用 CPU
    print("Using CPU")


Using GPU: NVIDIA GeForce MX450


In [22]:
# 加载 csv文件
data = pd.read_csv('merged_data.csv').drop(['AQI','PM2.5','PM10','NO2','CO','SO2','O3_8h'],axis=1)

# 检查结果
data

,日期,气温,气压,平均海平面大气压,气压趋势,湿度,风向,平均风速,最大阵风,天气情况,最低气温,最高气温,露点温度,降水时间,季节,质量等级
0,2013-12-02,3.162,759.712,764.462,-0.013,45.875,7.375,1.375,7.177,5.500,6.896,18.301,-8.588,10.717,冬,轻度污染
1,2013-12-03,5.488,761.725,766.425,0.100,39.000,8.000,1.625,7.177,4.750,7.209,18.314,-8.900,10.717,冬,良
2,2013-12-04,5.250,760.300,764.988,-0.138,45.375,9.375,1.250,7.177,1.750,7.134,18.714,-6.675,10.717,冬,轻度污染
3,2013-12-05,6.150,763.275,767.975,0.250,30.000,6.875,2.250,7.177,3.875,7.759,18.551,-10.912,10.717,冬,良
4,2013-12-06,2.925,760.325,765.075,-0.275,52.750,4.875,1.250,7.177,1.000,7.121,18.239,-6.350,10.717,冬,中度污染
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2023-12-26,-2.050,770.638,773.938,0.925,54.125,8.250,1.500,7.147,0.500,-9.475,2.588,-10.862,12.000,冬,良
3677,2023-12-27,-3.888,771.538,774.850,-0.538,67.750,7.250,1.125,4.250,0.000,-8.900,5.348,-9.450,12.000,冬,良
3678,2023-12-28,-3.012,769.138,772.438,-0.038,69.875,6.625,1.000,3.750,0.375,-9.100,3.750,-8.288,12.000,冬,轻度污染
3679,2023-12-29,-2.800,765.112,768.400,-0.938,78.125,5.625,1.125,4.147,2.000,-6.302,3.975,-6.300,12.000,冬,轻度污染


In [29]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# 初始化标签编码器
le = LabelEncoder()

# 对质量等级进行编码
data['质量等级'] = le.fit_transform(data['质量等级'])
data['季节'] = le.fit_transform(data['季节'])

# 计算选定特征列的相关性
correlation = data[selected_features].corr()['质量等级']

# 按照绝对值降序排序
correlation = correlation.reindex(correlation.abs().sort_values(ascending=False).index)

# 创建一个自定义的颜色映射
cmap = LinearSegmentedColormap.from_list('my_cmap', ['blue', 'white', 'red'])

# 绘制条形图
plt.figure(figsize=(10, 10))
correlation.plot(kind='barh', color=[cmap(i) for i in range(len(correlation))])

# 添加标题和标签
plt.title('Correlation with Quality Level')
plt.xlabel('Correlation')
plt.ylabel('Features')

# 显示图表
plt.show()

NameError: name 'LinearSegmentedColormap' is not defined